In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 56.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.6/447.6 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.9 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 56.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 46.6 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 54.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 35.9 MB/s eta 0:00:00


In [4]:
import pandas as pd

In [5]:
# Step 1: Load the dataset
# Update the paths with your file locations
train_data_path = "train.csv"
text_data_path = "test.csv"

# Load the datasets
try:
    train_df = pd.read_csv(train_data_path)
    text_df = pd.read_csv(text_data_path)
  #  train_df = pd.read_csv(train_data_path).sample(1500000, random_state=42)  
  # text_df = pd.read_csv(text_data_path).sample(70000, random_state=42)
    
    print("Datasets loaded successfully!")
    
    # Displaying a preview of the datasets
    print("Train Dataset Sample:")
    print(train_df.head())

    print("\nText Dataset Sample:")
    print(text_df.head())
    
except FileNotFoundError as e:
    print(f"Error: {e}")


Datasets loaded successfully!
Train Dataset Sample:
   id                                               text  toxicity  \
0   0  This is so cool. It's like, 'would you want yo...  0.000000   
1   1  Thank you!! This would make my life a lot less...  0.000000   
2   2  This is such an urgent design problem; kudos t...  0.000000   
3   3  Is this something I'll be able to install on m...  0.000000   
4   4               haha you guys are a bunch of losers.  0.893617   

   severe_toxicity  obscene  threat   insult  identity_attack  sexual_explicit  
0         0.000000      0.0     0.0  0.00000         0.000000              0.0  
1         0.000000      0.0     0.0  0.00000         0.000000              0.0  
2         0.000000      0.0     0.0  0.00000         0.000000              0.0  
3         0.000000      0.0     0.0  0.00000         0.000000              0.0  
4         0.021277      0.0     0.0  0.87234         0.021277              0.0  

Text Dataset Sample:
   id              

In [6]:
text_df.shape

(97320, 2)

In [7]:
train_df.shape

(1804874, 9)

In [8]:
# Optionally drop null or long entries
train_df = train_df.dropna(subset=['text'])
train_df = train_df[train_df['text'].str.len() <= 1000]

In [9]:
# Check label distribution
print(train_df[['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit']].sum())

toxicity           185933.109257
severe_toxicity      8270.111023
obscene             25046.614174
threat              16805.671660
insult             146470.335207
identity_attack     40854.609440
sexual_explicit     11922.950046
dtype: float64


# Data Preprocessing

In [10]:
import re

# Function to clean text data
def clean_text(text):
    if not isinstance(text, str):  # Check if the text is a string
        return ""
    # Remove non-alphanumeric characters (retain only letters and spaces)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Apply text cleaning to both datasets
if 'text' in train_df.columns:
    train_df['text'] = train_df['text'].apply(clean_text)
else:
    print("Column 'text' not found in train.csv")

if 'text' in text_df.columns:
    text_df['text'] = text_df['text'].apply(clean_text)
else:
    print("Column 'text' not found in test.csv")

# Display cleaned data samples
print("Cleaned Train Dataset Sample:")
print(train_df.head())

print("\nCleaned Test Dataset Sample:")
print(text_df.head())


Cleaned Train Dataset Sample:
   id                                               text  toxicity  \
0   0  this is so cool its like would you want your m...  0.000000   
1   1  thank you this would make my life a lot less a...  0.000000   
2   2  this is such an urgent design problem kudos to...  0.000000   
3   3  is this something ill be able to install on my...  0.000000   
4   4                haha you guys are a bunch of losers  0.893617   

   severe_toxicity  obscene  threat   insult  identity_attack  sexual_explicit  
0         0.000000      0.0     0.0  0.00000         0.000000              0.0  
1         0.000000      0.0     0.0  0.00000         0.000000              0.0  
2         0.000000      0.0     0.0  0.00000         0.000000              0.0  
3         0.000000      0.0     0.0  0.00000         0.000000              0.0  
4         0.021277      0.0     0.0  0.87234         0.021277              0.0  

Cleaned Test Dataset Sample:
   id                            

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [12]:
# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


# Custom Dataset

In [13]:
import torch
from torch.utils.data import Dataset

class CommentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        # Ensure compatibility with both Pandas and NumPy objects
        if isinstance(texts, pd.Series):
            self.texts = texts.reset_index(drop=True)
        else:
            self.texts = texts  # Assume it's already indexable (e.g., NumPy array or list)
        
        if isinstance(labels, pd.Series):
            self.labels = labels.reset_index(drop=True)
        else:
            self.labels = labels  # Assume it's already indexable (e.g., NumPy array or list)
        
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        try:
            # Fetch the text and label at the given index
            text = self.texts[idx]
            label = self.labels[idx]

            # Tokenize the text
            tokens = self.tokenizer(
                text, 
                max_length=self.max_length, 
                padding='max_length', 
                truncation=True, 
                return_tensors="pt"
            )

            return {
                'input_ids': tokens['input_ids'].squeeze(0),
                'attention_mask': tokens['attention_mask'].squeeze(0),
                'labels': torch.tensor(label, dtype=torch.float)
            }
        except Exception as e:
            print(f"Error at index {idx}: {e}")
            raise e


In [14]:
# !pip install ipywidgets --upgrade

In [15]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import torch

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [17]:
# Tokenizer using TorchText
from transformers import AutoTokenizer  # Hugging Face tokenizer for simplicity
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", vocab_size=5000)  # Using tokenizer, not pre-trained model

In [18]:
# Prepare Data
train_df = train_df.reset_index(drop=True)
X = train_df['text']
y = train_df[['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit']].values


In [19]:
!pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 16.8 MB/s eta 0:00:00


# TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Assuming 'train_df' contains the training dataset
# Extract features and target variables
X = train_df['text']  # Text data
y = train_df[['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit']]  # Labels

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)  # Transform text into numerical vectors
X_tfidf = tfidf.fit_transform(X)

# Convert TF-IDF features to a DataFrame (optional)
# xX_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Check the resulting DataFrame
print("TF-IDF feature matrix:")
# print(X_tfidf.head())

# Labels remain unchanged
print("Target variable sample:")
print(y.head())


TF-IDF feature matrix:
Target variable sample:
   toxicity  severe_toxicity  obscene  threat   insult  identity_attack  \
0  0.000000         0.000000      0.0     0.0  0.00000         0.000000   
1  0.000000         0.000000      0.0     0.0  0.00000         0.000000   
2  0.000000         0.000000      0.0     0.0  0.00000         0.000000   
3  0.000000         0.000000      0.0     0.0  0.00000         0.000000   
4  0.893617         0.021277      0.0     0.0  0.87234         0.021277   

   sexual_explicit  
0              0.0  
1              0.0  
2              0.0  
3              0.0  
4              0.0  


# Dataloader

In [21]:
y = train_df[['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit']].values

print('y.shape[1]',y.shape[1])
# Ensure consistent sizes (no need to align lengths manually, as there's no SMOTE-induced imbalance)
print(f"Length of X: {len(X)}")
print(f"Length of y: {len(y)}")

# Binarize multi-label targets if necessary
#print(f"y shape before binarization: {y.shape}")
#mlb = MultiLabelBinarizer()
#y = mlb.fit_transform(y)
#print(f"y shape after binarization: {y.shape}")

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a function for batched tokenization
def tokenize_batch(texts, tokenizer, max_length=128):
    """Tokenize a batch of texts."""
    tokens = tokenizer(
        texts,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    return tokens["input_ids"], tokens["attention_mask"]

# Create Dataset and DataLoader
train_dataset = CommentDataset(X_train, y_train, tokenizer, max_length=128)
val_dataset = CommentDataset(X_val, y_val, tokenizer, max_length=128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

print("DataLoaders are ready!")

y.shape[1] 7
Length of X: 1804859
Length of y: 1804859
DataLoaders are ready!


# Updated forward Method in the LSTM Model
Modify the forward method of your LSTMClassifier to apply mean pooling over the sequence outputs, ensuring that the model outputs exactly 7 values.

In [22]:
import torch.nn as nn
import torch

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels, num_layers, dropout_rate):
        super(LSTMClassifier, self).__init__()
        
        # Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # LSTM Layer
        self.lstm = nn.LSTM(
            embed_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout_rate
        )

        # Normalization
        self.ln = nn.LayerNorm(hidden_dim * 2)
        
        # Attention Layer
        self.attention = nn.Linear(hidden_dim * 2, hidden_dim * 2)  # Self-attention layer
        self.attention_weights = nn.Linear(hidden_dim * 2, 1)  # Attention weights computation
        
        # Dropout Layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully Connected Layer
        self.fc = nn.Linear(hidden_dim * 2, num_labels)
        
        # Sigmoid Activation
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None):
        # Embedding Layer
        embedded = self.embedding(input_ids)  # Shape: [batch_size, seq_length, embed_dim]
        
        # LSTM Layer
        lstm_out, _ = self.lstm(embedded)  # Shape: [batch_size, seq_length, hidden_dim * 2]

        # Apply Layer Normalization
        lstm_out = self.ln(lstm_out)
        
        # Attention Mechanism
        # Calculate attention scores
        attn_scores = torch.tanh(self.attention(lstm_out))  # Shape: [batch_size, seq_length, hidden_dim * 2]
        attn_weights = torch.softmax(self.attention_weights(attn_scores), dim=1)  # Shape: [batch_size, seq_length, 1]
        
        # Compute the context vector
        context = torch.sum(attn_weights * lstm_out, dim=1)  # Shape: [batch_size, hidden_dim * 2]
        
        # Dropout
        context = self.dropout(context)
        
        # Fully Connected Layer
        out = self.fc(context)  # Shape: [batch_size, num_labels]
        
        # Sigmoid Activation
        return self.sigmoid(out)
    
   

# Model Initialization

In [23]:
# Model Hyperparameters
vocab_size = tokenizer.vocab_size
embed_dim = 200  # Embedding dimension
hidden_dim = 256  # Hidden layer size in LSTM
num_labels = y.shape[1]  # Number of output labels
num_layers = 3  # Reduce layers as attention reduces the need for deeper stacks
dropout_rate = 0.5  # Dropout rate for regularization

# Model Initialization
model = LSTMClassifier(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_dim=hidden_dim,
    num_labels=num_labels,
    num_layers=num_layers,
    dropout_rate=dropout_rate
).to(device)


# Model training 

In [24]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import f1_score

#  Learning Rate Scheduler -----------------------------------------------------------------
# scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5, verbose=True)

def compute_metrics(outputs, labels):
    """
    Compute F1-score for multi-label classification.
    Args:
        outputs: Model's predicted probabilities (continuous values between 0 and 1).
        labels: Ground truth labels (binary values 0 or 1).
    Returns:
        Weighted F1-score.
    """
    # Threshold the predictions to convert probabilities to binary
    preds = (outputs > 0.4).float()  # Convert probabilities to binary predictions (0 or 1)

    # Ensure shapes are compatible
    if preds.dim() > 2:
        preds = preds.squeeze()  # Remove extra dimensions if present
    if labels.dim() > 2:
        labels = labels.squeeze()  # Remove extra dimensions if present

    # Convert to NumPy arrays
    labels = labels.cpu().numpy().astype(int)  # Cast labels to integer type
    preds = preds.cpu().numpy()

    # Debugging: Check shapes and data types
    # print(f"Labels shape: {labels.shape}, Predictions shape: {preds.shape}")
    # print(f"Labels type: {labels.dtype}, Predictions type: {preds.dtype}")

    # Ensure the shapes match
    if labels.shape != preds.shape:
        raise ValueError(f"Shape mismatch: Labels shape {labels.shape}, Predictions shape {preds.shape}")

    # Compute F1-score
    f1 = f1_score(labels, preds, average="weighted", zero_division=0)
    return f1



# Training loop -----------------------------------------------------------------------------------
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, patience=3):
    best_val_loss = float('inf')
    counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()

            # Gradient Clipping to prevent exploding gradients during backpropagation
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_loss/len(train_loader)}")

# Validation Step -----------------------------------------------------------------
        model.eval()
        val_loss = 0
        val_f1 = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                # Forward pass
                outputs = model(input_ids, attention_mask)
                
                # Compute loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                # Compute F1-score
                val_f1 += compute_metrics(outputs, labels)
        
        # Average validation metrics
        val_loss /= len(val_loader)
        val_f1 /= len(val_loader)
        print(f"Validation Loss: {val_loss}, Validation F1-Score: {val_f1}")

        # Scheduler calling -----------------------------------------------------------------
        scheduler.step(val_loss)

        # Early Stopping ------------------------------------------------------------------------------------------
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), '5_12_best_model.pt')  # Save best model
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered")
                break


In [25]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = nn.BCELoss() 

# Define optimizer AdamW
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)

# Define the learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5, verbose=True)


# Train the model
train_model(
    model=model,  # Your model instance
    train_loader=train_loader,  # Training data loader
    val_loader=val_loader,  # Validation data loader
    criterion=criterion,  # Loss function
    optimizer=optimizer,  # Optimizer
    scheduler=scheduler,  # Learning rate scheduler
    num_epochs=10,  # Total number of epochs
    patience=2  # Early stopping patience
)


Epoch 1/10, Training Loss: 0.0975636530576633
Validation Loss: 0.09342526657007168, Validation F1-Score: 0.020238253539082412
Epoch 2/10, Training Loss: 0.09307151174394085
Validation Loss: 0.09289755528567514, Validation F1-Score: 0.02051221978086924
Epoch 3/10, Training Loss: 0.0920022297384367
Validation Loss: 0.09303225609367174, Validation F1-Score: 0.019726448858579545
Epoch 4/10, Training Loss: 0.09145476368593786
Validation Loss: 0.0931353644927504, Validation F1-Score: 0.02040622847546301
Early stopping triggered


In [115]:
# Trail for different threshold To find best threshold

In [116]:
import numpy as np
from sklearn.metrics import f1_score

def tune_threshold(model, data_loader):
    thresholds = np.arange(0.1, 1.0, 0.1)  # Test thresholds from 0.1 to 0.9
    best_threshold = 0.5
    best_f1 = 0.0

    for threshold in thresholds:
        total_preds = []
        total_labels = []

        model.eval()
        with torch.no_grad():
            for batch in data_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                # Forward pass
                outputs = model(input_ids, attention_mask)

                # Apply current threshold
                predictions = (outputs > threshold).float()

                # Collect predictions and labels
                total_preds.append(predictions.cpu().numpy())
                total_labels.append(labels.cpu().numpy())

        # Flatten lists
        total_preds = np.vstack(total_preds)
        total_labels = np.vstack(total_labels)

        # Ensure both are binary (int type)
        total_preds = total_preds.astype(int)  # Convert predictions to int
        total_labels = total_labels.astype(int)  # Ensure labels are also int

        # Calculate F1-score
        f1 = f1_score(total_labels, total_preds, average="weighted", zero_division=0)
        print(f"Threshold: {threshold}, F1-score: {f1:.4f}")

        # Update best threshold
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    print(f"Best Threshold: {best_threshold}, Best F1-score: {best_f1:.4f}")
    return best_threshold

# Tune threshold
best_threshold = tune_threshold(model, val_loader)


Threshold: 0.1, F1-score: 0.0163
Threshold: 0.2, F1-score: 0.0250
Threshold: 0.30000000000000004, F1-score: 0.0332
Threshold: 0.4, F1-score: 0.0407
Threshold: 0.5, F1-score: 0.0464
Threshold: 0.6, F1-score: 0.0452
Threshold: 0.7000000000000001, F1-score: 0.0306
Threshold: 0.8, F1-score: 0.0079
Threshold: 0.9, F1-score: 0.0000
Best Threshold: 0.5, Best F1-score: 0.0464


# validation accuracy

In [26]:
import torch

# Function to calculate accuracy
def calculate_accuracy(model, data_loader, threshold=0.6):
    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask)

            # Apply threshold to predictions (e.g., 0.5 for binary classification)
            predictions = (outputs > threshold).float()

            # Calculate number of correct predictions
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.numel()

    # Calculate accuracy
    accuracy = total_correct / total_samples
    return accuracy

# Calculate and print the validation accuracy
val_accuracy = calculate_accuracy(model, val_loader)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


Validation Accuracy: 86.97%


# Generate predictions for the test dataset (kaggle submission)

In [27]:
import pandas as pd
import torch

# Step 1: Define the prediction function
def generate_predictions(model, tokenizer, test_texts, max_length=256):

    model.eval()
    predictions = []
    with torch.no_grad():
        for text in test_texts:
            # Tokenize the input text
            tokens = tokenizer(
                text,
                max_length=max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )
            input_ids = tokens["input_ids"].to(device)
            attention_mask = tokens["attention_mask"].to(device)

            # Get model predictions
            outputs = model(input_ids, attention_mask)  # Shape: [batch_size, 7]
            probabilities = outputs.squeeze().cpu().numpy()  # Convert to NumPy array
            
            # Convert probabilities to binary predictions
            binary_predictions = (probabilities > 0.5).astype(int)
            predictions.append(binary_predictions.tolist())
    return predictions

# Step 2: Load test dataset and generate predictions
test_texts = text_df['text'].tolist()  # Extract text data from the test dataset
test_ids = text_df['id'].tolist()  # Extract IDs from the test dataset

# Generate predictions using the trained model
test_predictions = generate_predictions(model, tokenizer, test_texts)


In [28]:
# Step 3: Create the submission DataFrame
submission_columns = ['id', 'toxicity', 'severe_tox', 'obscene', 'threat', 'insult', 'identity_att', 'sexual_explicit']
submission_df = pd.DataFrame(test_predictions, columns=submission_columns[1:])  # Add predictions
submission_df.insert(0, 'id', test_ids)  # Add the `id` column from the test dataset

# Step 4: Save predictions to a CSV file
submission_file_path = '5_12_submission.csv'
submission_df.to_csv(submission_file_path, index=False)
print(f"Submission file saved to {submission_file_path}")



Submission file saved to 5_12_submission.csv
